In [ ]:
import pandas as pd
import numpy as np

# Function to calculate synthetic ATM
def calculate_synthetic_atm(strike_price):
    return round(strike_price / 100) * 100

# Function to select straddle range
def select_straddle_range(synthetic_atm):
    return [synthetic_atm - 200, synthetic_atm - 100, synthetic_atm, synthetic_atm + 100, synthetic_atm + 200]

# Function to calculate EMA
def calculate_ema(data, period):
    return data.ewm(span=period, adjust=False).mean()

# Function to implement trailing stop loss (TSL)
def apply_tsl(strategy_data, tsl_type):
    trailing_sl = None
    profits = []
    lots = 5

    for index, row in strategy_data.iterrows():
        # Apply the TSL logic based on tsl_type
        if tsl_type == 2:
            if row['Profit'] >= 20:
                trailing_sl = row['EntryPrice'] + 20
            if row['Profit'] >= 40:
                trailing_sl = row['EntryPrice'] + 40
            if row['Profit'] >= 60:
                trailing_sl = row['EntryPrice'] + 60
            if row['Profit'] >= 80:
                trailing_sl = row['EntryPrice'] + 80

        elif tsl_type == 3:
            if row['Profit'] >= 20:
                lots -= 1
                profits.append(row['Profit'])
                if lots == 0:
                    break
        elif tsl_type == 4:
            # Implement logic for TSL4
            pass

    return profits

# Function to check entry rules
def check_entry(row):
    return row['close'] < row['EMA_15']

# Load data (replace with your dataset file path)
data = pd.read_csv('/Users/mdnadeem/Documents/Python/Quant Internship/NIFTY_2023-08-10.csv')

# Ensure correct column parsing
data['datetime'] = pd.to_datetime(data['datetime'])
data['SyntheticATM'] = data['strike_price'].apply(calculate_synthetic_atm)
data['StraddleRange'] = data['SyntheticATM'].apply(select_straddle_range)

# Calculate EMA band
data['EMA_15'] = calculate_ema(data['close'], 15)
data['EMA_25'] = calculate_ema(data['close'], 25)
data['EMA_35'] = calculate_ema(data['close'], 35)
data['EMA_45'] = calculate_ema(data['close'], 45)
data['EMA_55'] = calculate_ema(data['close'], 55)

# Define strategy parameters
indicator = 24500  # Example value, replace with actual indicator data
vwap = data['VWAP'].iloc[-1]  # Use the last VWAP value from the data

# Initialize columns
strategy_data = pd.DataFrame()
strategy_data['Timestamp'] = data['timestamp']
strategy_data['StrikePrice'] = data['strike_price']
strategy_data['Entry'] = data.apply(check_entry, axis=1)  # Apply row-by-row entry check

# Define EntryPrice based on Entry condition
strategy_data['EntryPrice'] = data['close'].where(strategy_data['Entry'], np.nan)

strategy_data['Exit'] = False  # Add your exit logic here
strategy_data['Profit'] = data['close'] - strategy_data['EntryPrice']  # Example profit calculation, adjust as needed

# Apply TSL and analyze performance
profits = apply_tsl(strategy_data, tsl_type=2)

# Display results
print(f"Profits: {profits}")

# Save results to a file
strategy_data.to_csv('/Users/mdnadeem/Documents/Python/Quant Internship/Output file/strategy_results.csv', index=False)
